In [1]:
import ibmcloudsql
import getpass
import urllib.parse
import numpy as np
import pandas as pd
import folium 


In [2]:
apikey=getpass.getpass('Api key') or apikey

#Enter your instance crn
instancecrn_encoded=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instancecrn_encoded

#Decode the crn 
instancecrn=urllib.parse.unquote(instancecrn_encoded)

#Enter your target url 
targeturl=input('Enter target URL for SQL results (leave empty to use ): ') or targeturl

sqlClient = ibmcloudsql.SQLQuery(apikey, instancecrn, targeturl, client_info='SQL Query Particulate Matter Data Exploration and Cleansing Notebook')
sqlClient.logon()
print('Your SQL Query web console link:')
sqlClient.sql_ui_link() 

Api key········
Enter SQL Query Instance CRN (leave empty to use previous one): crn%3Av1%3Abluemix%3Apublic%3Asql-query%3Aus-south%3Aa%2F262f34e831a941d5a6377d1084360669%3A293f7c09-aebd-4f86-8066-d875fceba4e3%3A%3A
Enter target URL for SQL results (leave empty to use ): cos://eu-geo/btw2019datasciencechallenge-donotdelete-pr-e7goqv7q3lrwjf
Your SQL Query web console link:
https://sql.ng.bluemix.net/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/262f34e831a941d5a6377d1084360669:293f7c09-aebd-4f86-8066-d875fceba4e3::


# Get all sensors from Berlin
lat 52.520008
lon 13.404954

In [3]:
sql_sds="""
WITH prefiltered AS 
     (SELECT s.sensor_id sensor_id, s.lon lon, s.lat lat, MAX(s.lon) max_lon, MAX(s.lat) max_lat , s.location location,
     min(timestamp) as start, max(timestamp) as end
      FROM cos://us-geo/sql/oklabdata/parquet/sds011/* STORED AS PARQUET s
      WHERE s.lat between -90 and 90 and s.lon between -180 and 180
      GROUP BY s.sensor_id, s.lon, s.lat, s.location
     )
SELECT sensor_id as sds011_sensor_id, ST_Distance(ST_Point(max_lon, max_lat), ST_Point(12.375089, 51.340349)) AS distance , location as location, max_lon as lon, max_lat as lat,
start as sds011_start, end as sds011_end
FROM prefiltered 
WHERE ST_Distance(ST_Point(max_lon, max_lat), ST_Point(13.404954, 52.520008)) <= 10000.0 
ORDER BY distance asc"""


sql_dht="""
WITH prefiltered AS 
     (SELECT s.sensor_id sensor_id, s.lon lon, s.lat lat, MAX(s.lon) max_lon, MAX(s.lat) max_lat , s.location location,
            min(timestamp) as start, max(timestamp) as end
      FROM cos://us-geo/sql/oklabdata/parquet/dht22/* STORED AS PARQUET s
      WHERE s.lat between -90 and 90 and s.lon between -180 and 180
      GROUP BY s.sensor_id, s.lon, s.lat, s.location
     )
SELECT sensor_id as dht22_sensor_id, ST_Distance(ST_Point(max_lon, max_lat), ST_Point(12.375089, 51.340349)) AS distance , location as location, max_lon as lon, max_lat as lat,
        start as dht22_start, end as dht22_end
FROM prefiltered 
WHERE ST_Distance(ST_Point(max_lon, max_lat), ST_Point(13.404954, 52.520008)) <= 10000.0 
ORDER BY distance asc"""

sds011_sensors = sqlClient.run_sql(sql_sds)
dht22_sensors = sqlClient.run_sql(sql_dht)

In [4]:
sds011_sensors
dht22_sensors

,dht22_sensor_id,distance,location,lon,lat,dht22_start,dht22_end
0,17962,139805.965540,9106,13.364,52.438,2018-10-30T21:48:59.000Z,2018-12-31T23:57:25.000Z
1,3268,139829.633572,1646,13.326,52.451,2017-06-05T21:05:51.000Z,2018-12-31T23:55:25.000Z
2,17932,140100.354907,9091,13.316,52.457,2018-10-30T11:29:35.000Z,2018-12-31T23:58:39.000Z
3,2669,140126.270245,1344,13.323,52.455,2017-05-30T15:59:15.000Z,2018-12-31T23:59:41.000Z
4,7204,140554.132042,3643,13.321,52.460,2017-11-20T20:04:43.000Z,2018-12-31T23:57:33.000Z
5,1848,140642.158998,924,13.357,52.449,2017-04-21T14:42:32.000Z,2018-12-31T23:58:53.000Z
6,5707,140741.252444,2878,13.305,52.467,2017-09-15T22:35:15.000Z,2018-12-31T23:59:44.000Z
7,2086,140817.681719,1048,13.320,52.463,2017-04-30T11:44:03.000Z,2018-03-18T16:25:38.000Z
8,18856,140845.804675,9557,13.302,52.469,2018-11-29T13:43:37.000Z,2018-12-31T23:41:17.000Z
9,11521,141141.259085,5817,13.324,52.465,2018-03-25T16:48:06.000Z,2018-06-02T12:31:19.000Z


# Welche Sensoren sind geeignete Kandidaten?

Wir benötigen Sensoren / Locations mit folgenden Eigenschaften:
* verfügen über Luftfeuchtigkeitssensor
* möglichst schon seit längerem Zeitraum verfügbar
* an der Bismarkstr / B2
* in der Nähe des Ernst-Reuter-Platz

In [41]:
both = pd.merge(sds011_sensors, dht22_sensors, on = 'location', how='left', sort = False, suffixes=('_sds011', '_dht22'))
both = both[['location','dht22_sensor_id', 'sds011_sensor_id', 'distance_sds011', 'sds011_start', 'lon_sds011', 'lat_sds011']]



In [62]:
%%capture
import folium 
from folium.features import DivIcon

lat = 52.520008
lon = 13.404954

erp_lat = 52.512628
erp_lon = 13.321881

# Create a map
m = folium.Map(location=[erp_lat, erp_lon], tiles='Stamen Terrain', zoom_start=80)

folium.Circle([erp_lat, erp_lon],
              radius = 5000,
              color = 'red'
             ).add_to(m)

folium.Marker([erp_lat, erp_lon],
              popup='Some Other Location',
              icon=folium.Icon(color='blue',icon='home')
              ).add_to(m)

for index, row in both.iterrows():
    folium.Marker(
    [row['lat_sds011'], row['lon_sds011']],
   icon = folium.Icon(color = 'gray'), popup = str(row['sds011_sensor_id'])
    ).add_to(m)
    if row['location'] in [10574, 6367, 8732, 3642, 3123]:
         folium.Marker([row['lat_sds011'], row['lon_sds011']],icon = folium.Icon(color = 'green')).add_to(m)
    if row['sds011_sensor_id'] in  [2101, 19683]:
        folium.Marker([row['lat_sds011'], row['lon_sds011']],icon = folium.Icon(color = 'red')).add_to(m)


    



In [64]:
m

In [65]:
m.save('berlin.html')